In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

In [4]:
ltcm = pd.read_excel('../data/ltcm_exhibits_data.xlsx', sheet_name=1,skiprows=2).rename(columns={"Unnamed: 0":
                                                                                                      "Date"})\
                                                                             .set_index('Date').dropna()
SPY = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name=2, index_col = 0)
rf = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name=3, index_col = 0)
SPY['rf'] = rf['US3M']
SPY_ex = (SPY['SPY']-SPY['rf']).to_frame('SPY Excess Return')
SPY_ex.head()

,SPY Excess Return
1993-02-28,0.0082
1993-03-31,0.0199
1993-04-30,-0.0281
1993-05-31,0.0244
1993-06-30,0.0011


In [5]:
ltcm.index = SPY_ex.loc['1994-03':'1998-07'].index
ltcm_ex = ltcm.subtract(SPY['rf'], axis = 0).dropna().drop(columns=['Index of Net Performance'])
ltcm_ex['SPY (excess)']=SPY_ex['SPY Excess Return']
ltcm_ex.columns = ['Fund Capital ($billions)', 'Gross Performance (excess)', 'Net Performance (excess)', 'SPY (excess)']
ltcm_ex.head()

,Fund Capital ($billions),Gross Performance (excess),Net Performance (excess),SPY (excess)
1994-03-31,1.0970,-0.0140,-0.0160,-0.0449
1994-04-30,1.0967,0.0107,0.0047,0.0079
1994-05-31,1.1964,0.0644,0.0494,0.0123
1994-06-30,1.1965,-0.0425,-0.0326,-0.0264
1994-07-31,1.3963,0.1123,0.0803,0.0287


# 2. LTCM Risk Decomposition

### 2.1 Summary Stats





In [6]:
def summary_stats(df, annual_fac=12):
    stat_df = (df.mean()*annual_fac).to_frame('Mean')
    stat_df['Vol'] = df.std()*np.sqrt(annual_fac)
    stat_df['Sharpe'] = stat_df['Mean']/stat_df['Vol']
    
    return stat_df

def tail_risk_stats(df, annual_fac=12):
    tr_df = df.skew().to_frame('Skew')
    tr_df['Kurtosis'] = df.kurtosis()
    tr_df['VaR (0.05)'] = df.quantile(0.05)
    return tr_df

(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.)

In [7]:
summary_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])


,Mean,Vol,Sharpe
Gross Performance (excess),0.2421,0.1362,1.7769
Net Performance (excess),0.1554,0.1118,1.3901
SPY (excess),0.1738,0.1123,1.5479


(b) Report the skewness, kurtosis, and (historic) VaR(.05).

In [8]:
tail_risk_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])

,Skew,Kurtosis,VaR (0.05)
Gross Performance (excess),-0.2877,1.5866,-0.0304
Net Performance (excess),-0.8102,2.9269,-0.0264
SPY (excess),-0.4335,-0.3620,-0.0464


(c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

- Gross excess return of LTCM has higher mean and similar volatility with SPY. In general, LTCM outperforms SPY with higher Sharpe ratio than SPY and lower tail risks. 
- The difference between gross and net excess return is not large. However, the sharpe ratio of net excess performance is a bit lower than SPY's.

### 2.2 Using the series of net LTCM excess returns, denoted $\widetilde{r}^{LTCM}$, estimate the following regression:


> $$\widetilde{r}^{LTCM} = \alpha +\beta^m\widetilde{r}_t^{m}+\epsilon_t$$



In [9]:
def regress(y, X, intercept = True, annual_fac=12):
    if intercept == True:
        X_ = sm.add_constant(X)
        reg = sm.OLS(y, X_).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['$R^2$'] = reg.rsquared
        reg_df.loc['const'] *= annual_fac
    else:
        reg = sm.OLS(y, X).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['$R^2$'] = reg.rsquared
    
    return reg_df

(a) Report $α$ and $β_m$. Report the $R^2$ stat.

In [10]:
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex['SPY (excess)']) 

,Regression Parameters
const,0.1315
SPY (excess),0.1371
$R^2$,0.0190


(b) From this regression, does LTCM appear to be a “closet indexer”?

Since beta is small, the low correlation between market LTCM and market is also low and LTCM's portfolio is much different from the market. Then LTCM cannot be a closet indexer.

(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

The constant is 0.1315 and beta is positive which show that LTCM appear to deliver excess returns largely beyong the risk premium that we expect from market exposure.

### 2.3 Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns:
> $$\widetilde{r}^{LTCM} = \alpha +\beta_1\widetilde{r}_t^{m}+\beta_2(\widetilde{r}_t^{m})^2+\epsilon_t$$

(a) Report $β_1$, $β_2$, and the $R^2$ stat.

In [11]:
ltcm_ex['SPY (excess) squared'] = ltcm_ex['SPY (excess)'] **2
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','SPY (excess) squared']])  

,Regression Parameters
const,0.1550
SPY (excess),0.1669
SPY (excess) squared,-1.9267
$R^2$,0.0243


(b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

The R squared only increases 0.0053 after including quadratic market factor regession compareed to single factor regression. Thus, the quandratic market factor does not increase the overall LTCM vairationn explained by the market that much.

(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

LTCM's negative market exposure to quadratic market factor behaves like shorting market options.

(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

LTCM is negatively exposed to market volatility with negative quadratic exposure.

### 2.4 Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:

> $\widetilde{r}_t^{LTCM} = \alpha +\beta\widetilde{r}_t^{m}+\beta_u max(\widetilde{r}_t^{m}-k_1,0)+\beta_d max(k_2-\widetilde{r}_t^{m},0)+\epsilon_t$

where k1 = .03 and k2 = −.03. (This is roughly one standard deviation of r ̃m.)


(a) Report $β$, $β_u$, $β_d$, and the $R^2$ stat.

In [12]:
#clip is threshold which limit the boundary to a and b in clip(a,b)
ltcm_ex['Up'] = (ltcm_ex['SPY (excess)'] - 0.03).clip(0) 
ltcm_ex['Down'] = (-0.03 - ltcm_ex['SPY (excess)']).clip(0)
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','Up','Down']])

,Regression Parameters
const,0.1012
SPY (excess),0.4666
Up,-0.7821
Down,1.2896
$R^2$,0.0555


(b) Is LTCM long or short the call-like factor? And the put-like factor?

LTCM short the call-like factor and long the put-like factor.

(c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

The put-like factor's coefficient has the largest absolute value or magnitude so it moves LTCM more

(d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?

Since LTCM is negatively exposed to market volatility, the volatility exposeure comes mroe from short the market's upside.

# 3.The FX Carry Trade

### Read Data

In [14]:
rf = pd.read_excel('../data/fx_carry_data.xlsx', sheet_name = 1).set_index('DATE')

log_rf = np.log(1 + rf)

log_rf.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3458e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3226e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4270e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8953e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.4997e-05


In [15]:
fx = pd.read_excel('../data/fx_carry_data.xlsx', sheet_name = 2).set_index('DATE')

log_fx = np.log(fx)

log_fx.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


### 3.1 The Static Carry Trade

In [16]:
log_rf_ex = log_rf.subtract(log_rf['USD1M'], axis=0)[log_rf.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()

log_rf_ex_col.columns = log_fx.columns

ret = (log_rf_ex_col.shift() + log_fx.diff()).dropna()

ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160
1999-06-30,-0.0157,-0.0128,-0.0211,-0.0045


For each foreign currency, i, calculate the excess log return series, ˜rt+1. Report the following
stats, (based on the excess log returns.) Annualize them.

In [17]:
summary_stats(ret)

,Mean,Vol,Sharpe
USUK,-0.0035,0.0863,-0.0406
USEU,-0.0044,0.0947,-0.0459
USSZ,0.0043,0.0988,0.0437
USJP,-0.0174,0.0915,-0.1903


What differences do you see across currencies?

The volatility does not have much differences across currencies. The mean returns are very small and USUK, USEU, and USJP even have negative mean return. Across those pairs, USD and JPY has the largest sharpe ratio which implies its largest magnitude in trading comparing to other currencies. 

### 3.2 Implications for UIP

(a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

Yes, some of these stats do contradict the log version of the Uncovereed Interest Parity. The USJP has a mean return of -0.0174 which is quite different from zero. Thus, it violates the UIP. For other currencies, their mean returns are very close to zero, so we can treat them as zero, or they can be treated as evidence of contradicting the UIP. 

(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?

The best trade would be short JPY and long USD.

(c) Are there any foreign currencies for which a long position earned a negative excess return
(in USD) over the sample?

Yes, the long position of UK, EU, and JP currencies will earn a negative excess return in USD. In the table, they have negative mean return. 

### 3.3 Predicting FX

(a)

In [18]:
ys = log_fx.diff().dropna()
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = [r'$\alpha^{i}$',r'$\beta^{i}$',r'$R^{2}$'])

for i in range(0,len(ys.columns)):
    fx_reg_pred[ys.columns[i]] = regress(ys[ys.columns[i]],Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
$\alpha^{i}$,-0.0059,0.0070,0.0436,-0.0060
$\beta^{i}$,0.4858,-1.2564,-1.6466,0.3715
$R^{2}$,0.0004,0.0026,0.0039,0.0005


(b) Suppose the foreign risk-free rate increases relative to the US rate.

i. For which foreign currencies would we predict a relative strengthening of the USD in
the following period?

UK and JP currency will be predicted to have a relative strengthening of the USD in the following period

ii. For which currencies would we predict relative weakening of the USD in the following
period?

EU and SZ currency will be predicted to have a relative weakening of the USD in the following period

iii. This FX predictability is strongest in the case of which foreign currency?

This FX predictability is strongest in the case of SZ currency. 

### 3.4 Dynamtic Carry Trade

In [19]:
alphas = (fx_reg_pred.loc[r'$\alpha^{i}$'] / 12)
betas = fx_reg_pred.loc[r'$\beta^{i}$']
Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas - 1) * Xs)

ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030


(a)

In [20]:
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0])/len(ret_ex_forecast)) * 100

ret_ex_forecast

month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0


(b) Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?

SZ seems to show consistent positive FX risk premium (calculated at above 63% of the months). On the other hand, JP's FX risk premium is negative more than 99.9% of the time in the data: making it the currency that most often goes negative.

On a separate note, despite the fact that section 3.3 found UK and EU having negative mean excess (log) returns, the result in 3.4 (a) shows that they are positive bout 23.8% and 50.2%, respectively, with dynamic carry trade.

(c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

We can use the forecasted excess returns to compute the weights of each currency trade in our portfolio. The resulting trading strategy from a dynamic carry trade would be better than the one built on static carry trade.

In other words, we would be able to dynamically change the magnitude and direction of the currency trades based on forecast rather than being passively invested in a currency based on evidence given by the static carry trade analysis.

Nevertheless, the effectiveness and profitability of this strategy is still questionable due to the low R-squared value: even with an in-sample prediction.